In [1]:
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
import altair as alt

2025-10-02 10:33:22.555485: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)
print(tf.config.list_physical_devices('GPU'))

2.20.0
3.11.3
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Latihan Membangun Sistem Rekomendasi

## impor library

In [3]:
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
import altair as alt

In [5]:
alt.data_transformers.enable('default', max_rows=None)

DataTransformerRegistry.enable('default')

Kode di atas memiliki dua buah sintaks yang berperan sebagai *ground rules* ketika menggunakan **Altair** sebagai library visualisasi data. Mari kita jabarkan satu per satu:

### 1. `alt.data_transformers.enable('default', max_rows=None)`
Baris ini mengatur *data transformer* Altair untuk digunakan dalam notebook. *Data transformer* ini mengubah data mentah menjadi format yang dapat digunakan oleh Altair untuk membuat visualisasi.  

- `default` → mengatur transformer ke mode bawaan yang cocok untuk visualisasi.  
- `max_rows=None` → menghilangkan batasan jumlah baris data yang dapat diplot.  
  - Ini bermanfaat jika kita memiliki dataset besar yang ingin divisualisasikan sepenuhnya.  

---

### 2. `alt.renderers.enable('colab')`
Baris ini mengatur Altair agar dapat digunakan dalam **Google Colab**.  

- *Renderer* adalah komponen yang bertanggung jawab menampilkan visualisasi di lingkungan kerja (misalnya browser atau notebook).  
- `colab` → mengatur renderer agar visualisasi Altair dapat ditampilkan langsung di Google Colab.  

Dengan pengaturan ini, visualisasi yang kita buat menggunakan Altair akan muncul secara interaktif di Google Colab.


## mengunduh dataset

In [7]:
print("Downloading movielens data...")
from urllib.request import urlretrieve
import zipfile
 
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "./bahan/movielens.zip")
zip_ref = zipfile.ZipFile('./bahan/movielens.zip', "r")
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


In [8]:
# Load masing masing dataset (users, movies, and ratings).
users_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(
    './bahan/ml-100k/u.user', sep='|', names=users_cols, encoding='latin-1')
 
ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    './bahan/ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')
# Genre pada setiap film bersifat biner.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    './bahan/ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

In [9]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [11]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [12]:
print('Jumlah data film: ', len(movies.movie_id.unique()))
print('Jumlah data pengunjung: ', len(users.user_id.unique()))
print('Jumlah data penayangan: ', len(ratings.user_id))

Jumlah data film:  1682
Jumlah data pengunjung:  943
Jumlah data penayangan:  100000


Sampai di sini, apakah Anda melihat sesuatu yang kurang tepat pada dataset di atas? Yup, hal tersebut terdapat pada fitur user_id dan movie_id yang dimulai dari angka satu. Agar dapat memudahkan proses perhitungan dan menyelaraskan dengan format komputasi yang ada, mari kita ubah geser urutannya dimulai dari nol. Selain itu, kita juga perlu memisahkan tahun rilis dari masing-masing film yang kelak akan digunakan pada proses pelatihan. Untuk melakukan kedua proses tersebut, gunakanlah kode di bawah ini.

In [13]:
users["user_id"] = users["user_id"].apply(lambda x: str(x-1))
movies["movie_id"] = movies["movie_id"].apply(lambda x: str(x-1))
movies["year"] = movies['release_date'].apply(lambda x: str(x).split('-')[-1])
ratings["movie_id"] = ratings["movie_id"].apply(lambda x: str(x-1))
ratings["user_id"] = ratings["user_id"].apply(lambda x: str(x-1))
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

Sebagai tahapan eksplorasi data, alangkah baiknya jika kita juga mengetahui jumlah genre yang ada sebagai informasi tambahan ketika melakukan modelling khususnya collaborative filtering.

In [14]:
genre_occurences = movies[genre_cols].sum().to_dict()
genre_occurences

{'genre_unknown': 2,
 'Action': 251,
 'Adventure': 135,
 'Animation': 42,
 'Children': 122,
 'Comedy': 505,
 'Crime': 109,
 'Documentary': 50,
 'Drama': 725,
 'Fantasy': 22,
 'Film-Noir': 24,
 'Horror': 92,
 'Musical': 56,
 'Mystery': 61,
 'Romance': 247,
 'Sci-Fi': 101,
 'Thriller': 251,
 'War': 71,
 'Western': 27}

Lalu, apakah Anda masih ingat bahwa genre dari masing-masing film yang ada berupa data biner? Hal tersebut akan menyulitkan proses pelatihan karena nantinya kita akan melakukan proses embeddings seperti pada modul Natural Language Processing sebelumnya. Oleh karena itu, kita perlu membuat sebuah fungsi yang dapat menggabungkan genre menjadi satu buah string. 

In [15]:
def mark_genres(movies, genres):
  def get_random_genre(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return np.random.choice(active)
  def get_all_genres(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return '-'.join(active)
  movies['genre'] = [
      get_random_genre(gs) for gs in zip(*[movies[genre] for genre in genres])]
  movies['all_genres'] = [
      get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]
 
mark_genres(movies, genre_cols)

In [16]:
movies["all_genres"]

0       Animation-Children-Comedy
1       Action-Adventure-Thriller
2                        Thriller
3             Action-Comedy-Drama
4            Crime-Drama-Thriller
                  ...            
1677                        Drama
1678             Romance-Thriller
1679                Drama-Romance
1680                       Comedy
1681                        Drama
Name: all_genres, Length: 1682, dtype: object

In [17]:
movielens = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

In [19]:
movielens.shape

(100000, 34)

In [20]:
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]
 
def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df
 
pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

Secara keseluruhan, kode tersebut memiliki dua tujuan utama yaitu sebagai berikut:

1. **Pengaturan Opsi Tampilan**  
   Bagian pertama dari kode, yaitu `pd.options.display.max_rows` dan `pd.options.display.float_format` bertujuan untuk mengatur tampilan default saat kita mencetak atau menampilkan DataFrame menggunakan library **pandas**.  
   - Dengan mengatur `max_rows`, kita membatasi jumlah baris yang akan ditampilkan secara default.  
   - Dengan `float_format`, kita mengatur format tampilan untuk nilai floating point dalam DataFrame.  

2. **Penambahan Fungsi-Fungsi Manipulasi DataFrame**  
   Bagian kedua dari kode mendefinisikan dua fungsi, yaitu `mask` dan `flatten_cols`, dan menambahkannya ke objek DataFrame dari library **pandas**. Ini memperluas fungsionalitas bawaan DataFrame dengan fungsi-fungsi baru yang dapat digunakan untuk manipulasi data, yaitu:  
   - **mask** → untuk memfilter DataFrame berdasarkan kriteria yang diberikan pada kolom tertentu.  
   - **flatten_cols** → untuk menggabungkan multi-indeks kolom pada DataFrame menjadi satu level sehingga mempermudah manipulasi dan analisis data.  


In [21]:
users.describe(include="all")

,user_id,age,sex,occupation,zip_code
count,943,943.000,943,943,943
unique,943,NaN,2,21,795
top,0,NaN,M,student,55414
freq,1,NaN,670,196,9
mean,NaN,34.052,NaN,NaN,NaN
...,...,...,...,...,...
min,NaN,7.000,NaN,NaN,NaN
25%,NaN,25.000,NaN,NaN,NaN
50%,NaN,31.000,NaN,NaN,NaN
75%,NaN,43.000,NaN,NaN,NaN


In [22]:
# Membuat filter untuk melakukan slicing data.
occupation_filter = alt.selection_point(fields=["occupation"])
occupation_chart = alt.Chart().mark_bar().encode(
    x="count()",
    y=alt.Y("occupation:N"),
    color=alt.condition(
        occupation_filter,
        alt.Color("occupation:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).properties(width=300, height=300, selection=occupation_filter)
 
# Fungsi yang dapat membuat histogram dari data yang sudah difilter.
def filtered_hist(field, label, filter):
  base = alt.Chart().mark_bar().encode(
      x=alt.X(field, bin=alt.Bin(maxbins=10), title=label),
      y="count()",
  ).properties(
      width=300,
  )
  return alt.layer(
      base.transform_filter(filter),
      base.encode(color=alt.value('lightgray'), opacity=alt.value(.7)),
  ).resolve_scale(y='independent')

In [24]:
occupation_filter = alt.selection_multi(fields=["occupation"])
occupation_chart = alt.Chart().mark_bar().encode(
    x="count()",
    y=alt.Y("occupation:N"),
    color=alt.condition(
        occupation_filter,
        alt.Color("occupation:N", scale=alt.Scale(scheme='category20')),
        alt.value("lightgray")),
).add_selection(occupation_filter) # Move selection here
# ... (Rest of your code remains the same)
# Create a chart for the count, and one for the mean.
alt.hconcat(
    filtered_hist('rating count', '# ratings / user', occupation_filter),
    filtered_hist('rating mean', 'mean user rating', occupation_filter),
    occupation_chart,
    data=ratings)

/tmp/ipykernel_5307/1727027246.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  occupation_filter = alt.selection_multi(fields=["occupation"])
/tmp/ipykernel_5307/1727027246.py:9: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(occupation_filter) # Move selection here


ValueError: Unable to determine data type for the field "rating count"; verify that the field name is not misspelled. If you are referencing a field from a transform, also confirm that the data type is specified correctly.

alt.HConcatChart(...)